## BERT Based Sentiment Analysis Model Server
The model used here, was trained with the concept of transfer learning  i.e. taking huggingface transformers pretrained BERT model and further training it on a custom dataset of reviews. this yields a sentiment analysis model based on the prior knowledge of BERT. 
The model server is given a list of texts and outputs a list of labels corresponding to its prediction.
The labels express the sentiment of the writer towards the topic of the text:
0 for negative sentiment, 1 for neutral and 2 for positive.

The model file (~430 MB), can be downloaded to your local environment from: https://iguazio-sample-data.s3.amazonaws.com/models/model.pt

In [ ]:
# define imports
import os
import wget
import mlrun
from mlrun import import_function

In [ ]:
# declare global variables
MODEL_PATH = os.path.join(os.path.abspath('./'), 'models')

In [7]:
# Run this to download the pre-trained model to your `models` directory
def download_pretrained_model(model_path):
    model_location = 'https://iguazio-sample-data.s3.amazonaws.com/models/model.pt'
    # Create paths
    os.makedirs(model_path, exist_ok=1)
    model_filepath = os.path.join(model_path, os.path.basename(model_location))
    !wget -nc -P {saved_models_directory} {model_location} 

File ‘/User/myfunctions/functions/sentiment_analysis_serving/models/model.pt’ already there; not retrieving.



In [8]:
model = MODEL_PATH+'/model.pt'
    if not path.exists(model):
        download_pretrained_model(MODEL_PATH)
    fn = import_function('hub://sentiment_analysis_serving')
    fn.add_model('mymodel', model_path=model, class_name='SentimentClassifierServing')
    # create an emulator (mock server) from the function configuration)
    server = fn.to_mock_server()



> 2021-03-15 14:38:21,799 [info] model model1 was loaded
> 2021-03-15 14:38:21,800 [info] Loaded ['model1']


#### example 1
Here we test a pretty straightforward example for positive sentiment.

In [9]:
instances = ['I had a pleasure to work with such dedicated team. Looking forward to \
                 cooperate with each and every one of them again.']
result = server.test("/v2/models/mymodel/infer", {"instances": instances})
result

[2]


#### example 2
Now we will test a couple more examples. These are arguably harder due to misleading words that express, on their own, an opposite sentiment comparing to the full text. 

In [10]:
instances = ['This app is amazingly.', 
                     'As much as I hate to admit it, the new added feature is surprisingly user friendly.']
result2 = server.test("/v2/models/mymodel/infer", {"instances": instances})
result2

[0, 2]


### remote activation
Create a function object with custom specification.

In [ ]:
addr = fn.deploy(project='bert-example')